In [ ]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
from collections import OrderedDict


In [ ]:
import risk_portfolios as p

from risk_portfolios import construct_equal_weight_portfolio, construct_inverse_volatility_portfolio, construct_tracking_portfolio, construct_tracking_with_penalty, construct_fixed_weight_portfolio


# Initialize parameters

In [ ]:
# INITIALIZATION


# Define the full list of tickers
#tickers = ['SPY', 'IEF', 'QQQ', 'IWM', 'EEM']
tickers = [ 'MWTIX', 'SPY', 'IWM', 'MDY', 'RSP', 'QQQ', 'XLK', 'XLI', 'XLF', 'XLC', 'XLE', 'XLY', 'XLB', 'XLV', 'XLU', 'XLP', 'VNQ', 'AIQ', 'ICLN', 'PFF', 'FEZ', 'EEM', 'FXI', 'ASHR',  'LQD', 'HYG', 'LQDH', 'HYGH', 'AGG',  'SHY', 'IEI', 'IEF', 'TLT', 'TIP', 'VTIP', 'AGNC', 'VMBS', 'CMBS', 'EMB', 'EMHY', 'GLD', 'SLV', 'USO', 'DBC', 'UUP', 'FXE', 'FXY' ]
x_tickers=[ 'AGG', 'IEF', 'VMBS', 'IEI', 'LQD', 'TLT', 'TIP', 'SHY', 'EMB']

#************ assert x_tickers is a subset of ticker
# Define the start and end dates
end_date = datetime.today()
start_date = end_date - timedelta(days=30 * 365)

# Download the adjusted closing prices for all tickers
# price_data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']
price_data = yf.download(tickers, start=start_date, end=end_date)['Close'] # API Changed

# Calculate daily returns - *************************************   fix to account for diffrent bus days carry forward
returns = price_data.pct_change().dropna()

# Generate month-end rebalancing dates using 'ME'

rebalancing_dates = returns.resample('M').last().index # ME is deprecated

# Initialize a DataFrame to store portfolio weights in long format
portfolio_weights_long = pd.DataFrame(columns=["portfolio_name", "date", "ticker", "weight"])

# Set half-life for exponential weighting (6 months ~ 126 trading days)
halflife = 126
min_periods = 60  # Minimum number of data points required

# Set the lag parameter (number of business days)
lag = 1  # Adjust as needed call it   ******** execution delay




# Portfolio Construction

# Map function names to actual functions
portfolio_functions = {
    "EW": construct_equal_weight_portfolio,
    "VW": construct_inverse_volatility_portfolio,
    "TRACK": construct_tracking_portfolio,
    "TRACK_PENALTY": construct_tracking_with_penalty,
    "FIXED": construct_fixed_weight_portfolio,
}

# Prepare the fixed weights data
fixed_weights_data = pd.DataFrame({
    'date': pd.to_datetime(['2015-01-31', '2024-10-31', '2024-06-30']),
    'MWTIX': [1, 0.5, 0.75],
    'SHY': [0, 0.5, 0.1],
    # Add other tickers as needed
})

# Set the date as index for easier lookup
fixed_weights_data.set_index('date', inplace=True)



# Define the portfolios using an OrderedDict
portfolios = OrderedDict({
    "TCW": {
        "function_to_call": "FIXED",
        "ticker_subset": ['MWTIX',  'SHY'],
        "other_options": {
            "fixed_weights_data": fixed_weights_data,
        },
    },
    # Include other portfolios as before
    "Vol_wtd_ptfl": {
        "function_to_call": "VW",
        "ticker_subset": x_tickers,
        "other_options": {
            "halflife": halflife,
            "min_periods": min_periods,
            "lag": lag,
        },
    },
    "EW_ptfl": {
        "function_to_call": "EW",
        "ticker_subset": x_tickers,
    },
    "Tracking_ptfl": {
        "function_to_call": "TRACK",
        "ticker_subset": x_tickers,
        "other_options": {
            "target_portfolio_name": 'MWTIX',
            # Initial weights will be set dynamically
        },
    },

    "Tracking_ptfl_penalty": {  # New tracking portfolio with penalty
        "function_to_call": "TRACK_PENALTY",
        "ticker_subset": x_tickers,
        "other_options": {
            "target_portfolio_name": 'MWTIX',
            "penalty_weight": 1e0,  # Set penalty parameter
            # Initial weights will be set dynamically
        },
    },
})



# Run portfolio construction

In [ ]:
from importlib import reload
reload(p)

portfolio_tuple = p.build_all_portfolios(
    returns,
    rebalancing_dates,
    portfolio_functions,
    portfolios,
    halflife,
    min_periods,
    lag,
    fixed_weights_data,
    portfolio_weights_long,
)
returns, portfolio_weights_long = portfolio_tuple

# Observe output

In [ ]:
returns.iloc[:, -5:].cumsum().plot()


In [ ]:
import xarray as xr
xr.set_options(keep_attrs=True,
               display_expand_data=False)
import plotly.express as px
from risk_chart import px_format

def draw_portfolio_weights(portfolio_weights_long):
    ser = portfolio_weights_long.set_index(['date', 'portfolio_name', 'ticker'])
    da = ser.to_xarray()['weight']

    fig = (px.imshow(da.transpose(),
                     facet_col='portfolio_name',
                     facet_col_wrap=1,
                     height=1200,
                     zmin=-1,
                     zmax=+1,
                     color_continuous_scale='RdBu',
                     template='plotly_white')
           .for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1].strip()))
           .for_each_annotation(lambda a: a.update(text='') if a.x == 0 else None)
           .update_layout(coloraxis_showscale=False,
                          xaxis_title=None,
                          yaxis_title=None))
    return fig

fig = draw_portfolio_weights(portfolio_weights_long)
fig